<img style="float: center;" src='https://github.com/STScI-MIRI/MRS-ExampleNB/raw/main/assets/banner1.png' alt="stsci_logo" width="1000px"/> 

# Specviz2D tutorial - JWebbinar 32
-----

**Author**: Ricky O'Steen | Data Analysis Tools Branch, Space Telescope Science Institute
<br>
**Last Updated**: May 10th, 2024
<br>

In this tutorial, you will learn how to extract a one dimensional spectrum from a two dimensional spectrum using
Specviz2D, and see how you can perform some basic analysis of the extracted spectrum. The first thing we'll do is
import the packages that we'll be using in the notebook below:

In [ ]:
import astropy.units as u
from glue.core.roi import XRangeROI
from jdaviz import Specviz2d

## A brief detour: data retrieval from MAST

The data we will be using in this tutorial is a spectrum of a planetary nebula with strong emission lines that was
used to [verify the NIRSpec wavelength calibration](https://www.stsci.edu/jwst/phase2-public/1125.pdf),
and can be retrieved from [MAST](https://archive.stsci.edu/). Data in MAST
can be opened in JDaviz in your web browser if it is in a format compatible with one of the Jdaviz configurations.
You can see an example of this with the data that we'll be using by clicking [here](https://mast.stsci.edu/viz/ui/#/spectra?uri=mast%3AJWST%2Fproduct%2Fjw01125-o001_s00001_nirspec_f290lp-g395h-s200a1-subs200a1_s2d.fits). All of the UI features
that we'll see today are available through MAST, with the downside of not being able to use notebook cells for
Jdaviz API calls or additional analysis with other tools. 

In the top right of the Jdaviz-in-MAST page linked, above you should see a button labeled "Open Data In". Clicking that
will allow you to select "Jupyter", which will download a notebook to your local machine with code in it to download
the data you were looking at on MAST and open it in Jdaviz, giving you a local copy of what you were seeing on the page.
Note that any operations or analyis you did on the web version will not be reflected in this local notebook, the downloaded
notebook simply retrieves the data and opens Jdaviz in a fresh state. The commented-out code block below to retrieve the
data from MAST is what you would see if you downloaded one of these notebooks from MAST, but today we have the data
preloaded locally on the jwebbinar platform.

In [ ]:
'''
import os
from astroquery.mast import Observations

# Get authentication token and login
auth_token = os.environ.get('MAST_API_TOKEN', None)
Observations.login(auth_token)

# download the data file
uri = f"mast:jwst/product/jw01125-o001_s00001_nirspec_f290lp-g395h-s200a1-subs200a1_s2d.fits"
result = Observations.download_file(uri, base_url='https://mast.stsci.edu/api/v0.1/Download/file')
if result[0] == 'ERROR':
    raise RuntimeError('Error retrieving file: ' + result[1])

# construct the local filepath to load into Jdaviz
local_path = os.path.join(os.path.abspath('.'), uri.rsplit('/', 1)[-1])
'''

In [ ]:
local_path = "/home/shared/preloaded-fits/jdaviz_data/jw01125-o001_s00001_nirspec_f290lp-g395h-s200a1-subs200a1_s2d.fits"

## Loading data into Specviz2D

Now we load the data file and display the Jdaviz application. Note that you can also call the `specviz2d.show()` method
before loading data, and then load data either through the API or by clicking the `Import Data` button in the top left
of the Jdaviz application.

In [ ]:
specviz2d = Specviz2d()
specviz2d.load_data(local_path)
specviz2d.show()

## Spectral Extraction with Specviz2D

We'll be focusing on the Spectral Extraction plugin for the first part of the tutorial. Each plugin has an API that
can be accessed in code cells to interact with the plugin, in addition to using the UI. This can be useful for operations
that you may want to repeat every time you open a notebook, or simply to keep track of what you've done in your analysis.
To access the Spectral Extraction API, we run:

In [ ]:
extract = specviz2d.plugins["Spectral Extraction"]

Running `extract.open_in_plugin()` opens the plugin in the Jdaviz application.
To see all of the parameters and methods that are accessible through the API, you can run `dir(extract)`:

In [ ]:
extract.open_in_tray()
dir(extract)

Accessing the parameter associated with a selection menu will show you both the choices and which of those
choices is currently selected:

In [ ]:
extract.trace_type

Here, we will set the trace type to "Polynomial" and change the order from 3 to 5:

In [ ]:
extract.trace_type = "Polynomial"
extract.trace_order = 5

We will explore the remaining options in the UI. One important thing to note is that the preview marks
displayed in the viewers are context-dependent, based on which section of the plugin your mouse is hovering over.
If you are in the trace section, the central trace for the extraction is shown, whereas in the background section
the background windows are shown, and in the extract section the central trace and window width are shown.

After setting the parameters as desired, clicking the `Extract` button will create a new data entry in the 1D spectrum
viewer. If the data label chosen is the same as existing data, the button will instead read `Extract (Overwrite)`. The
trace itself and some quantities related to the background can also be exported as their own data entries, but as these
are less commonly used, these options are hidden in collapsed expandable sections.

The extracted spectrum can be retrieved into the notebook with the `get_data()` method:

In [ ]:
extracted_spec = specviz2d.get_data("Spectrum 1D")
extracted_spec

You could then analyze this spectrum using, for example, tools from `astropy`. Alternatively, some basic
analysis can be done from the Jdaviz application, for example measuring or fitting emission lines. You
can run the following to define a subset around one of the emission features in the spectrum, or use the UI
subset selection to highlight another feature of your choosing:

In [ ]:
sv = specviz2d.app.get_viewer('spectrum-viewer')
sv.toolbar_active_subset.selected = []
sv.apply_roi(XRangeROI(4.29, 4.31))

Opening the `Line Analysis` plugin and selecting the subset you just created under "Spectral region"
will automatically calculate some common measures of the line of interest. Alternatively, you can fit an
[astropy.modeling](https://docs.astropy.org/en/stable/modeling/) model from within the Jdaviz application
in the `Model Fitting` plugin for more a more customizable analysis.

<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/>